# Manufacturer Popularity Over Time

This notebook produces a charts of manufacturer popularity, based on number of sightings, over time:

- Absolute, based on number of sightings
- Relative share, based on share of annual sightings

Manufacturers with a number of sightings below a threshold are excluded.

To use the notebook, set the threshold value and export format for charts in the first code cell before running the notebook.

In [ ]:
# Annual sightings threshold below which manufacturers are excluded from the results
sightings_threshold = 10

# Export format for the charts:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
export_format = "PNG"

In [ ]:
from pathlib import Path
import sqlparse

# Read the query file
query_file_path = Path("sql") / "sightings.sql"
with open(query_file_path.absolute(), "r") as f:
    query = f.read().replace("\n", " ")

# Show a pretty-printed form of the query
print(sqlparse.format(query, reindent=True, keyword_case="upper"))

In [ ]:
import pandas as pd
import sqlite3
import os

# Connect to the database, execute the query and read the results into a dataframe
database_path = os.environ["FLIGHT_RECORDER_DB"]
connection = sqlite3.connect(database_path)
df = pd.read_sql_query(query, connection, parse_dates=["Date"])

# Check there is some data
if not df.shape[0]:
    message = f"No data found"
    raise ValueError(message)

In [ ]:
import pandas as pd

# Create a year column
df["Year"] = df["Date"].dt.year

# Display unique manufacturers
df["Manufacturer"].unique()

# Aggregate Sightings by Manufacturer and Year
manufacturer_trends = (
    df.groupby(["Year", "Manufacturer"])
    .size()
    .reset_index(name="Sightings")
)

# Remove manufacturers that fall below the sightings threshold
row_indices = manufacturer_trends[manufacturer_trends["Sightings"] < sightings_threshold].index
manufacturer_trends.drop(row_indices, inplace = True) 

# Preview the trend data
with pd.option_context("display.max_rows", None,
                       "display.max_columns", None,
                       "display.precision", 3,
                       ):
    display(manufacturer_trends)

In [ ]:
# Create a pivot table of absolute sighting counts per manufacturer per year
pivot_counts = (
    manufacturer_trends
    .pivot(index="Year", columns="Manufacturer", values="Sightings")
    .fillna(0)
    .sort_index()
)

# Normalize by total sightings per year
total_per_year = df.groupby("Year").size().rename("Total")
normalized = manufacturer_trends.merge(total_per_year, on="Year")
normalized["Share"] = normalized["Sightings"] / normalized["Total"]

# Create a pivot table of relative shares of sightings per manufacturer per year
pivot_shares = (
    normalized
    .pivot(index="Year", columns="Manufacturer", values="Share")
    .fillna(0)
    .sort_index()
    * 100
)

In [ ]:
import pandas as pd
from pathlib import Path

# Create the folder to hold exported reports
export_folder_path = Path("exported")
export_folder_path.mkdir(parents=True, exist_ok=True)

# Create a Pandas Excel writer
export_file_name = "Manufacturer-Popularity"
output_path = export_folder_path / f"{export_file_name}.xlsx"

with pd.ExcelWriter(output_path.absolute(), engine="openpyxl") as writer:
    pivot_counts.to_excel(writer, sheet_name="Absolute Counts", index=True)
    pivot_shares.to_excel(writer, sheet_name="Relative Share", index=True)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(14, 7))

# Plot the bar chart
pivot_counts.plot(
    kind="bar",
    stacked=True,
    ax=ax
)

# Set axis properties
ax.set_title("Aircraft Sightings by Manufacturer (Counts)")
ax.set_xlabel("")
ax.set_ylabel("Number of Sightings")

# Place the legend below the chart
ax.legend(
    title="Manufacturer",
    loc="upper center",
    bbox_to_anchor=(0.5, -0.12),
    ncol=3,
    frameon=False
)

plt.tight_layout()

# Export to PNG
if export_format.casefold() == "png":
    export_file_path = export_folder_path / f"{export_file_name}-Counts.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"{export_file_name}-Counts.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(14, 7))

# Plot the bar chart
pivot_shares.plot(
    kind="bar",
    stacked=True,
    ax=ax
)

# Set axis properties
ax.set_title("Aircraft Sightings by Manufacturer (Relative Share)")
ax.set_xlabel("")
ax.set_ylabel("Share of Sightings")
ax.set_ylim(0, 100)
ax.yaxis.set_major_formatter('{x:.0f}%')

# Place the legend below the chart
ax.legend(
    title="Manufacturer",
    loc="upper center",
    bbox_to_anchor=(0.5, -0.12),
    ncol=3,
    frameon=False
)

plt.tight_layout()

# Export to PNG
if export_format.casefold() == "png":
    export_file_path = export_folder_path / f"{export_file_name}-Share.png"
    plt.savefig(export_file_path.absolute(), format="png", dpi=300, bbox_inches="tight")

# Export to PDF
if export_format.casefold() == "pdf":
    export_file_path = export_folder_path / f"{export_file_name}-Share.pdf"
    plt.savefig(export_file_path.absolute(), format="pdf", bbox_inches="tight")

# Show the plot
plt.show()
